In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display   #for loading and visualizing audio files
import IPython.display as ipd   #to play audio

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import time 
import pickle

# Notes

Start writing up presentation as you go after modelling stage

## 3 methods:

**A.** PCA each feature (spectrogram) ---> save each PCA'd feature into own dataframe ---> model on each one individually --> majority vote predictions

**B.** From PCA'd dataframe combine into one big and model entirely ---> predictions

**C.** Do descriptive stats and model on this ---> predictions


- simple model on various features count predictions and choose from majority vote

(chroma, tempo, spectogram, mse)
- MAKE SURE each AUDIO file is same sample rate!


- turn into black & white images, pca

**N. of frames from hop length and y**

If you divide your y_shape length by the hop_length - you get the number of the frames. 

- y.shape/hop_length

**Regarding warning message :**

There is nothing to fix: this is operating as intended.

Again, this is not an error, it is a warning. Your code will work just fine, it just will fall back on the (slower) audioread decoder. This is unavoidable for now, since libsndfile still does not support mp3.

If you're really concerned about speed, you could transcode your mp3s into a more permissive format (eg ogg vorbis

In [ ]:
'''
chroma_stft([y, sr, S, norm, n_fft, …])            Compute a chromagram from a waveform or power spectrogram.

chroma_cqt([y, sr, C, hop_length, fmin, …])        Constant-Q chromagram

chroma_cens([y, sr, C, hop_length, fmin, …])       Computes the chroma variant “Chroma Energy Normalized” (CENS)

melspectrogram([y, sr, S, n_fft, …])               Compute a mel-scaled spectrogram.

mfcc([y, sr, S, n_mfcc, dct_type, norm, lifter])   Mel-frequency cepstral coefficients (MFCCs)

rms([y, S, frame_length, hop_length, …])           Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S.

spectral_centroid([y, sr, S, n_fft, …])            Compute the spectral centroid.

spectral_bandwidth([y, sr, S, n_fft, …])           Compute p’th-order spectral bandwidth.

spectral_contrast([y, sr, S, n_fft, …])            Compute spectral contrast

spectral_flatness([y, S, n_fft, hop_length, …])    Compute spectral flatness

spectral_rolloff([y, sr, S, n_fft, …])             Compute roll-off frequency.

poly_features([y, sr, S, n_fft, hop_length, …])    Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.

tonnetz([y, sr, chroma])                           Computes the tonal centroid features (tonnetz)

zero_crossing_rate(y[, frame_length, …])           Compute the zero-crossing rate of an audio time series.
'''

In [8]:
'''Yeah pretty much - Librosa turns the raw audio into a 1D numpy array with X numbers per second 
(you define the sample rate)
[9:29 AM] You can then use this array to compute all the other features, which take samples over this array
[9:30 AM] So at this point your music is a time series but you have 1 series per different feature
[9:30 AM] I just took the mean and standard deviation from all these features.
[9:31 AM] I think that’s part of the reason people use images. It’s a better way of capturing all the information 
and convolutional neural networks use a 2D filter rather than using SKLearn which would have to use a 1d numpy array
[9:31 AM] It someone else who wrote the paper, you can check it out here: 
https://qmro.qmul.ac.uk/xmlui/bitstream/handle/123456789/46029/CHOI_Keunwoo_PhD_Final_190918.pdf?sequence=1&isAllowed=y
[9:31 AM] Is someone planning on doing a music project?

Finn Alexander  [9:37 AM]
Congratulations, thats very exciting about CapGemini!
[9:39 AM] yeah two people are planning on doing music classification, seems to be all the rage!
[9:42 AM] Just so i understand you took the mean and std across the time series for each feature or frequency or 
did you take it across the frequencies? Did you try classifying using chroma frequencies as well as spectrograms. 
(edited) 
[9:42 AM] thanks for the link

Adam Shafi  [9:59 AM]
Thanks! I’m really excited!
[9:59 AM] Wow it is all the rage! Are they both using GTZAN?
[9:59 AM] Yes I took the mean and std across the time series for each feature
[9:59 AM] I used chroma, mfcc, tempo and many others
[10:00 AM] but even using all of those, using black and white images with dimensionality reduction and 
sklearn models performed way better

Finn Alexander  [11:10 AM]
Very interesting, I imagine you tried all of this convolutional networks stuff. Will have a look a t GTZAN and push 
them in that direction? (edited) 

Adam Shafi  [11:33 AM]
GTZAN is a dataset, Joh used it, its really popular and probably a good choice.
Using convolutional neural networks is pretty hard and on GTZAN you can get some good results without them. 
Take a look on Kaggle and there are some great notebooks!
''';

Old way to collect audio time series and create dataframe 

As it is lower to retrieve from pandas dataframe, decided to use pickle dump to store numpy array files on harddrive

In [ ]:
#get all filepaths for a given folder and store the audio time series y, sr = librosa.load(test_filepath, duration =25)

fp_main = '/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/'
folders = os.listdir(fp_main)

numbers = ['0','1','2','3','4','5','6','7','8','9']

track_id_list = []
audio_timeseries_list = []  #saved as a tuple (y,sr)
filepath_list = []
folder_list = []
error_list = []

#tranch this with finns code
for folder_numb in folders:
    if folder_numb == '101': # limit till what file number you want to collect. 
        break
    if any(x in folder_numb for x in numbers):  #to prevent picking up hidden files such as 'README.txt' or 'checksums'
        print("folder :",folder_numb)
        fp_main_new = fp_main + folder_numb + '/'  #not forget the '/' at the end
        audio_clips = os.listdir(fp_main_new)
        for track_id in audio_clips:
            if '_' in track_id:
                pass
            else:
                fp_new = fp_main_new + track_id    
                try:                #need try and except as y,sr extraction doesnt work for all e.g '001486.mp3'
                    #fp_new = fp_main_new + track_id              #fp for filepath
                    y,sr = librosa.load(fp_new, duration = 30) #clips are 30secs
    
                    track_id_list.append(track_id.split('.')[0])
                    audio_timeseries_list.append((y,sr)) 
                    error_list.append('none') 
                    filepath_list.append(fp_new) 
                    folder_list.append(folder_numb)
                except:
                    track_id_list.append(track_id.split('.')[0]) #track_id
                    audio_timeseries_list.append('error') #numpy array
                    error_list.append('error')
                    filepath_list.append(fp_new) 
                    folder_list.append(folder_numb)
    else:
        pass 

In [ ]:
df_time_series = pd.DataFrame({'track_id': track_id_list,'time_series' : audio_timeseries_list, 
                               'error_track' : error_list,'folder' : folder_list,'filepath' : filepath_list})

In [79]:
#alternative Getting summary statistic feature dictionaries for all numpy arrays stored in feature_dictionaries*** (*Do not run twice*
for feat_dict in feature_dict_0_50:
    track_id = feature_dict_0_50[i].split('_')[2].split('.')[0]
    
    load_feat_from = open('/Volumes/Extreme SSD/CAPSTONE_DATA/saved/features_dictionaries/{}'.format(feat_dict), "rb")
    a_feat_dict = pickle.load(load_feat_from)
    
    get_statistics_dict(a_feat_dict, reduce_limit = 50)
    
    save_ss_feat_dict_to = open('/Volumes/Extreme SSD/CAPSTONE_DATA/saved/ss_feature_dictionaries/dict_ss_feat_{}.pkl'.format(track_id), "wb")
    pickle.dump(dictionary, save_ss_feat_dict_to)
    save_ss_feat_dict_to.close()   
    

'dict_feat_000139.pkl'

In [59]:
df_time_series

,track_id,time_series,error_track,folder,filepath
0,000002,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
1,000003,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
2,000005,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
3,000010,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
4,000134,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
...,...,...,...,...,...
410,001965,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
411,001966,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
412,001967,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
413,001995,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
